In [22]:
#HIDDEN
from IPython import display
from ipywidgets import widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import randint
import math
from pandas.tseries.offsets import MonthEnd, MonthBegin
from pandas.plotting import register_matplotlib_converters
from IPython.core.display import display as display_core, HTML


register_matplotlib_converters()


def split_nfes_values(df, days_in_between, column):
    list_terms = [l for l in days_in_between if l is not None and l > 0]
    parcelas = []
    df[f"value_split_{column}"] = df[column]/len(list_terms)
    for e, d in enumerate(list_terms):
        df_local = df.loc[:, ["date", f"value_split_{column}"]]
        df_local["date"] = df_local["date"] + pd.Timedelta(days=days_in_between[e])
        parcelas.append(df_local)
        
    df_parcelas = pd.concat(parcelas)
    df_parcelas.fillna(0, inplace=True)
    
    
    return df_parcelas.groupby("date", as_index=False)[[f"value_split_{column}"]].sum()
    

def get_income(df):
    df.sort_values(["issued_at", "due_date"], inplace=True)
    df_issued = df.groupby("issued_at", as_index=False)[["value"]].sum()
    df_due = df.groupby("due_date", as_index=False)[["value"]].sum()
    df_issued.rename(columns={"issued_at": "date", "value": "issued_at_value"}, inplace=True)
    df_due.rename(columns={"due_date": "date", "value": "due_date_value"}, inplace=True)
    df_income = df_issued.merge(df_due, on="date", how="outer")
    df_income.fillna(0, inplace=True)
    
    return df_income

class Cashflow_simulation:
    
    def __init__(self):
        style = {'description_width': 'initial'}
        self.marging_industry_text = widgets.Text(description='Margem:', placeholder="2%", value="2%", style=style)
        self.income_per_year_text = widgets.IntText(description='Fat. Anual (K):', placeholder=15000, value=30000, style=style)
        self.num_employees_text = widgets.IntText(description='# Funcionários:', placeholder=80, value=80, style=style)
        self.avg_salary_text = widgets.IntText(description='Salário Médio (K):', placeholder=4, value=4, style=style)
        self.percentage_receivables_text = widgets.Text(description='% Fornecedores:', placeholder="35%", value="35%", style=style)
        self.cash_text = widgets.IntText(description='$ Caixa (K):', placeholder=1500, value=1500, style=style)
        self.months_text = widgets.IntText(description='Meses Simulação:', placeholder=12, value=12, style=style)
        
        self.avg_lenght_seller_1 = widgets.IntText(description='Parcela 1:', placeholder=22, value=22, style=style)
        self.avg_lenght_seller_2 = widgets.IntText(description='Parcela 2:', placeholder=35, value=35, style=style)
        self.avg_lenght_seller_3 = widgets.IntText(description='Parcela 3:', placeholder=47, value=47, style=style)
        self.avg_lenght_seller_4 = widgets.IntText(description='Parcela 4:', style=style)
        self.avg_lenght_seller_5 = widgets.IntText(description='Parcela 5:', style=style)
        
        self.avg_lenght_buyer_1 = widgets.IntText(description='Parcela 1:', placeholder=22, value=22, style=style)
        self.avg_lenght_buyer_2 = widgets.IntText(description='Parcela 2:', placeholder=35, value=35, style=style)
        self.avg_lenght_buyer_3 = widgets.IntText(description='Parcela 3:', placeholder=47, value=47, style=style)
        self.avg_lenght_buyer_4 = widgets.IntText(description='Parcela 4:', style=style)
        self.avg_lenght_buyer_5 = widgets.IntText(description='Parcela 5:', style=style)
        
        # Tempo de estoque em dias
        
        self.months_check = []
        self.month_boxs = []
        months = ["Jan", "Fev", "Mar", "Abr", "Mai", "Jun",
                 "Jul", "Ago", "Set", "Out", "Nov", "Dez"]
        
        for e, m in enumerate(months):
            self.months_check.append(widgets.Checkbox(description=f"{m}", value=False, indent=False))
            if (e + 1) % 3 == 0:
                self.month_box = widgets.VBox(self.months_check)
                self.month_boxs.append(self.month_box)
                self.months_check = []
            
            
        self.month_boxes = widgets.HBox(self.month_boxs)
        self.df = None
        
        self.out = widgets.Output()
        
        
        self.box_general_company = widgets.VBox([
             widgets.HTML("<center>Informações Empresa:"),
            self.marging_industry_text,
            self.num_employees_text,
            self.avg_salary_text,
            self.percentage_receivables_text,
            self.cash_text,
        ])
        
        self.box_simulation = widgets.VBox([
            widgets.HTML("<center><b>Informações p/ Simulação:"),
            widgets.HBox([
                widgets.VBox([
                    self.income_per_year_text,
                    self.months_text,    
                ]),
                widgets.VBox([widgets.HTML("<b>Meses com maior Faturamento:"), self.month_boxes])
            ])
        ])
        
        self.box_nfes_seller = widgets.VBox([
            widgets.HTML("<center><b>Parcelas pagamento Sacados:"),
            self.avg_lenght_seller_1,
            self.avg_lenght_seller_2,
            self.avg_lenght_seller_3,
            self.avg_lenght_seller_4,
            self.avg_lenght_seller_5,
        ])
        
        self.box_nfes_buyer = widgets.VBox([
            widgets.HTML("<center><b>Parcelas pagamento Fornecedores:"),
            self.avg_lenght_buyer_1,
            self.avg_lenght_buyer_2,
            self.avg_lenght_buyer_3,
            self.avg_lenght_buyer_4,
            self.avg_lenght_buyer_5,
        ])
        
        self.box = widgets.HBox([
            self.box_simulation
        ], layout=widgets.Layout(border='3px solid black')
        )
        
        from IPython.display import FileLink
        
        df = pd.DataFrame(columns=["data", "data faturamento", "data vencimento"])
        csv_file_name = "exemplo.csv"
        df.to_csv(csv_file_name, index=False)
        self.example_csv = FileLink(csv_file_name, result_html_prefix="Click here to download example ")
        
        
        self.uploader = widgets.FileUpload(accept= '.csv', multiple=False)
        self.button = widgets.Button(description="Click Me!")
        
        display.display(widgets.HBox([self.box_general_company, self.box_nfes_buyer, self.box_nfes_seller, self.uploader], 
                                     layout=widgets.Layout(border='3px solid black')),
                        self.example_csv,
#                         self.box,
                        self.button,
                        self.out
                       )

        def on_button_clicked(b):
            with self.out:
                self.on_button_clicked()

        self.button.on_click(on_button_clicked)

        
    def on_button_clicked(self):
        from io import BytesIO
        with self.out:
            df_upload = pd.DataFrame()
            self.out.clear_output()
            
            # Get Initial Input Values

            marging_industry = float(self.marging_industry_text.value.replace("%", "").strip())/100
            num_employees = self.num_employees_text.value
            avg_salary = self.avg_salary_text.value * 1.3 * num_employees # $ Annual total package / 12 months

            percentage_receivables = float(self.percentage_receivables_text.value.replace("%", "").strip())/100
            cash = self.cash_text.value
            months = self.months_text.value
            
            
            df_receivables = pd.read_csv(BytesIO(self.uploader.data[0]), sep=";", decimal=",")
            df_receivables["due_date"] = pd.to_datetime(df_receivables["due_date"])
            df_receivables["issued_at"] = pd.to_datetime(df_receivables["issued_at"])
            df_receivables["term"] = (df_receivables["due_date"] - df_receivables["issued_at"]).dt.days
            mean_term = (df_receivables["term"] * df_receivables["value"]).sum()/df_receivables["value"].sum()
            max_fat_date = df_receivables["issued_at"].max()
            df_upload = get_income(df_receivables.copy())
#             df_receivables.to_csv("output.csv", sep=";", decimal=",", index=False)
#             df_upload.query("date <= @max_fat_date", inplace=True)
            df_upload["issued_at_value"] /= 1000
            df_upload["due_date_value"] /= 1000
            total_revenue = df_upload["issued_at_value"].sum()
            
            df_upload.reset_index(inplace=True)
            df_upload["date"] = pd.to_datetime(df_upload["date"])
            df_upload["month"] = df_upload["date"].dt.strftime("%Y%m")
            months = df_upload.query("date <= @max_fat_date").month.nunique()
            
            df_upload["cost_total"] = ((df_upload["issued_at_value"]) / (1 + marging_industry))
            df_upload["cost_receivable"] = df_upload["cost_total"] * (percentage_receivables)
            

            df_salary = pd.DataFrame()
            df_salary["date"] = pd.to_datetime(df_upload.query("date <= @max_fat_date")["date"], format="%Y%m") + MonthEnd(1)
            df_salary.drop_duplicates("date", inplace=True)
            df_salary["salary"] = avg_salary
            
            df_other_expense = pd.DataFrame()
            df_other_expense["date"] = pd.to_datetime(df_upload.query("date <= @max_fat_date")["date"], format="%Y%m") + MonthBegin(1) + pd.Timedelta(days=5)
            df_other_expense.drop_duplicates("date", inplace=True)
            
            salary = avg_salary * months
            annual_other_expenses = max((df_upload["cost_total"] * (1 - percentage_receivables)).sum() - salary, 0)
            monthly_other_expenses = annual_other_expenses/months
            
            df_other_expense["other_expenses"] = monthly_other_expenses
            
            df_cashflow = df_upload.merge(df_salary, on="date", how="outer")
            df_cashflow = df_cashflow.merge(df_other_expense, on="date", how="outer")
            df_cashflow.fillna(0, inplace=True)
            df_cashflow.reset_index(inplace=True)
            df_cashflow.sort_values("date", inplace=True)

#             df_cashflow.to_csv("upload.csv", sep=";", decimal=",", index=False)
            
            days_in_between_buyer = [self.avg_lenght_buyer_1.value,
                               self.avg_lenght_buyer_2.value,
                               self.avg_lenght_buyer_3.value,
                               self.avg_lenght_buyer_4.value,
                               self.avg_lenght_buyer_5.value]
            
            days_in_between_seller = [self.avg_lenght_seller_1.value,
                               self.avg_lenght_seller_2.value,
                               self.avg_lenght_seller_3.value,
                               self.avg_lenght_seller_4.value,
                               self.avg_lenght_seller_5.value]
            
            df_split_receivables_cost = split_nfes_values(df_cashflow.copy(), days_in_between=days_in_between_buyer, column="cost_receivable")
            df_split_receivables_cost.rename(columns={f"value_split_cost_receivable": "receivable_cost"}, inplace=True)
            
            df_cashflow = df_cashflow.merge(df_split_receivables_cost, on="date", how="outer")
           
            df_cashflow.fillna(0, inplace=True)

            df_cashflow["cost"] = df_cashflow["other_expenses"] + df_cashflow["receivable_cost"] + df_cashflow["salary"]
            df_cashflow["cost_book"] = df_cashflow["other_expenses"] + df_cashflow["cost_receivable"] + df_cashflow["salary"]
            
            min_date = df_cashflow["date"].min() - MonthBegin(1)
            max_date = df_cashflow["date"].max() + MonthEnd(1)
            df_dates = pd.DataFrame(columns=["date"])
            df_dates["date"] = pd.date_range(end = max_date, start=min_date, freq="D")
            df_year = pd.concat([df_dates, df_cashflow], sort=True)
            df_year.fillna(0, inplace=True)
            df_year = df_year.groupby("date", as_index=False).sum()
            df_year.reset_index(inplace=True, drop=True)
            df_year["date"] = pd.to_datetime(df_year["date"]).dt.date

            df_year["cash"] = 0
            df_year.loc[0, "cash"] = cash
            df_year.fillna(0, inplace=True)
            
            df_year.sort_values("date", inplace=True)
            df_year["net"] = df_year["due_date_value"] + df_year["cash"] - df_year["cost"]
            df_year["net_book"] = df_year["issued_at_value"] + df_year["cash"] - df_year["cost_book"]
           
            df_year["net_cumsum"] = df_year["net"].cumsum()
            df_year["net_book_cumsum"] = df_year["net_book"].cumsum()

            total_cost = df_year["cost_book"].sum()
            total_revenue = df_year["issued_at_value"].sum()
            margin = total_revenue/total_cost - 1
            
            df_year["month"] = pd.to_datetime(df_year["date"]).dt.strftime("%Y%m")
            monthly_values = df_year.groupby("month", as_index=False)[["issued_at_value", "due_date_value", "cost", "other_expenses"]].sum()
            
            negative_months = monthly_values.query("due_date_value < cost")["month"].unique()
            df_overview = df_year.agg(['sum'])
            df_overview = df_overview.loc[:, ["issued_at_value", "cost", "cost_receivable", "salary", "other_expenses"]]
            df_overview.rename(columns={
                "issued_at_value": "Faturamento no Período",
                "cost": "Custo total no período",
                "cost_receivable": "Custo com Fornecedores",
                "salary": "Custo com Salário",
                "other_expenses": "Custo com outras despesas"
            }, inplace=True)
            
            df_overview["Margem"] = margin * 100
            df_overview["Prazo médio no período"] = round(mean_term)
            df_overview["Caixa inicial no período"] = cash
            df_overview["Caixa final no período"] = df_year["net_cumsum"].values[-1].astype(int)
            
            df_overview = df_overview.apply(round, axis=1)
            df_overview = df_overview.T
            
            df_overview.columns = ["Resumo da simulação"]
            df_overview["Resumo da simulação"] = df_overview["Resumo da simulação"].astype(int)
            
            display_core(HTML(df_overview.to_html()))
            
            NUM_ROWS = 2
            IMGs_IN_ROW = 2
            f, ax = plt.subplots(NUM_ROWS, IMGs_IN_ROW, figsize=(18,12))
            
            ax[0][0].plot(df_year["date"], df_year["net_cumsum"])
            ax[0][1].plot(df_year["date"], df_year["net_book_cumsum"])
            
            ax[1][0].bar(monthly_values["month"], monthly_values["issued_at_value"])
            ax[1][1].bar(monthly_values["month"], monthly_values["due_date_value"])

            ax[0][0].set_title('Available Cash Per Day - Cashflow')
            ax[0][1].set_title('Available Cash Per Day - Book Value')
            ax[1][0].set_title('Revenue')
            ax[1][1].set_title('Due Date - revenue')
            for ax in f.axes:
                ax.tick_params(labelrotation=45)
                
            

            title = 'Cashflow Comparison: Real x Book Value'
            f.suptitle(title, fontsize=16)
            plt.show()
            
            self.uploader.set_state({"_counter": 0})
            
            return df_year
        
cf = Cashflow_simulation()

/home/eduardo/Documentos/eduardo/simulator/exemplo.csv

Button(description='Click Me!', style=ButtonStyle())

Output()